In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

from lightgbm import LGBMClassifier

import joblib

RAND=42

# Evaluation

In [2]:
test_path = '../data/transformed/test_data/test_data'

test_data_00 = pd.read_parquet(test_path + '_0.pq.0')
test_data_01 = pd.read_parquet(test_path + '_0.pq.1')
test_data_10 = pd.read_parquet(test_path + '_1.pq.0')
test_data_11 = pd.read_parquet(test_path + '_1.pq.1')

In [3]:
models_path = "../data/models.joblib"

models = joblib.load(models_path)

model = models['lightgbm']

In [6]:
test_data = pd.concat([test_data_00, test_data_01,
                       test_data_10, test_data_11])
test_data = test_data.set_index("id")

In [7]:
y_score = model.predict_proba(test_data)

In [8]:
target_path = "../data/raw/test_target.csv"

test_id = pd.read_csv(target_path)
test_id = test_id.set_index('id')

In [9]:
test_id['score'] = y_score[:, 1]

In [11]:
sub_path = '../submission.csv'
test_id.to_csv(sub_path)